[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lsJaqWYR2bNaz2zt2XMTesJqZw64KMyh)

## SETUP

In [ ]:
from google.colab import output

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: Please use low quality or low res <--"
  print(warning_string)
  output.eval_js('alert("Warning - low GPU (see message)")')
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

print("Installing...")
from IPython.utils import io
with io.capture_output() as captured:
  !rm -Rf pixray
  !git clone --recursive https://github.com/pixray/pixray
  !pip install -r pixray/requirements.txt
  !pip uninstall -y tensorflow 
  !git clone https://github.com/pixray/diffvg
  %cd diffvg
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  !pip freeze | grep torch

import os
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu to continue!"
else:
  result_msg = "Setup Complete! Good luck with your drawing"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))

## PIXRAY

In [ ]:
#@title Easy Settings

#@markdown Let's take a prompt and try drawing with a number
#@markdown of different models. The models we will be using are
#@markdown VQGAN models which mainly differ on the trained data
#@markdown and architecture. Swapping the model will make the result
#@markdown look differnt. For example: "wikiart" is trained on
#@markdown artwork and so draws things less photorealistic.

#@markdown Best bet: just try different models out and see what happens.

import sys
sys.path.append("pixray")

prompts = "strange aura in the style of Theodor Severin Kittelsen" #@param {type:"string"}

vqgan_model = "wikiart_16384" #@param ["imagenet_f16_16384", "imagenet_f16_1024", "openimages_f16_8192", "wikiart_1024", "wikiart_16384", "coco"]

quality = "best" #@param ["draft", "normal", "better", "best"]

aspect = "widescreen" #@param ["widescreen", "square", "portrait"]

#@markdown When you have the settings you want, press the play button on the left.
#@markdown We will apply these settings below.

In [ ]:
# Simple setup from menu choices
import pixray
pixray.reset_settings()
pixray.add_settings(prompts=prompts)
pixray.add_settings(drawer="vqgan", vqgan_model=vqgan_model)
pixray.add_settings(quality=quality, aspect=aspect)
pixray.add_settings(display_clear=True)

# hey, let's try that new RN50x16 CLIP model huh?
pixray.add_settings(clip_models="RN50x16")

# examples below start with '#" are comments and do not get run
# pixray.add_settings(init_image="file.png")
# pixray.add_settings(init_image_alpha=200)
# pixray.add_settings(overlay_image="file.png")
# pixray.add_settings(image_prompts="file.png")
# pixray.add_settings(overlay_every=20)
# pixray.add_settings(seed=42)
# pixray.add_settings(display_every=50)
# pixray.add_settings(iterations=200)
# pixray.add_settings(size=[256, 256])
# pixray.add_settings(vector_prompts="textoff:4")

# if you enable drawers below make sure to disable the "vqgan_model" line above
# pixray.add_settings(drawer="fft")
# pixray.add_settings(drawer="pixel")
# pixray.add_settings(drawer="vdiff")

# change from CLIP guided to SLIP guided by enabling this line
# pixray.add_settings(perceptors="slip")